In [14]:
!pip install keras

In [15]:
!pip install tensorflow --user
!pip3 install tensorflowjs

In [16]:
import pandas as pd
import numpy as np
import keras as kr
import os
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.saved_model.save import save

from keras.optimizers import Adam

In [4]:
# Load data from CSV
df = pd.read_csv('malicious_phish - Copy.csv', on_bad_lines='skip')

In [5]:
#df = pd.read_csv('goPhish_featureExtracted.csv')

In [6]:
#input_shape = (651191, 22)

In [7]:
# Extract URLs and labels
urls = df['url'].tolist()
labels = df['type'].tolist()

# Tokenise and preprocess the URLs
tokeniser = Tokenizer()
tokeniser.fit_on_texts(urls)
sequences = tokeniser.texts_to_sequences(urls)

# Padding sequences to have the same length
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Convert labels to numpy array
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [8]:
#callback = kr.callbacks.EarlyStopping(monitor='accuracy')

# Model definition
model = Sequential()#Sequential model
model.add(Embedding(input_dim=len(tokeniser.word_index) + 1, output_dim=32, input_length=max_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid')) #activation function

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.1), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=2500, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/10


188/188 [==============================] - 319s 2s/step - loss: 0.6500 - accuracy: 0.6538 - val_loss: 0.6435 - val_accuracy: 0.6562
Epoch 2/10
188/188 [==============================] - 308s 2s/step - loss: 0.3470 - accuracy: 0.8455 - val_loss: 0.1865 - val_accuracy: 0.9323
Epoch 3/10
188/188 [==============================] - 314s 2s/step - loss: 0.1487 - accuracy: 0.9480 - val_loss: 0.1697 - val_accuracy: 0.9357
Epoch 4/10
188/188 [==============================] - 315s 2s/step - loss: 0.1102 - accuracy: 0.9621 - val_loss: 0.1615 - val_accuracy: 0.9401
Epoch 5/10
188/188 [==============================] - 315s 2s/step - loss: 0.0917 - accuracy: 0.9684 - val_loss: 0.1639 - val_accuracy: 0.9408
Epoch 6/10
188/188 [==============================] - 326s 2s/step - loss: 0.0837 - accuracy: 0.9716 - val_loss: 0.1644 - val_accuracy: 0.9419
Epoch 7/10
188/188 [==============================] - 327s 2s/step - loss: 0.0770 - accuracy: 0.9734 - val_loss: 0.1598 - val_accuracy: 0.9

In [9]:
# Make predictions on new data
new_urls = [input()]
new_sequences = tokeniser.texts_to_sequences(new_urls)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_length, padding='post')
predictions = model.predict(new_padded_sequences)

# Print the predictions
for url, prediction in zip(new_urls, predictions):
    print(f"{url} - Predicted Probability: {prediction[0]}")

facebook.com
1/1 [==============================] - 0s 209ms/step
facebook.com - Predicted Probability: 0.04133550822734833


In [18]:
save_dir = os.path.join('/GoPhish_Development/','goPhish/saved_goPhish.keras')
model.save(save_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/GoPhish_Development/goPhish/saved_goPhish.keras'